In [ ]:
!pip install scikit-learn

In [ ]:
from torchvision.models import resnet34, ResNet34_Weights
from torch import nn
import numpy as np
import os
import torch
import torchvision
from PIL import Image
SAVE_PATH='/content/drive/MyDrive/DR_grading/trained_models'
model=resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)
# 修改全连接层的输出
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 6)

model.load_state_dict(torch.load(SAVE_PATH+'/Resnet34_4epoch.pt'))
model.eval()
img_path='/content/drive/MyDrive/dataset/DDR-dataset/DR_grading/test/007-2523-100.jpg'
with torch.no_grad():
   img = Image.open(img_path) # fp -文件名 (字符串)，pathlib.Path对象或文件对象
   w, h = img.size
   slide=min(h,w)
   center_cropper=torchvision.transforms.CenterCrop(slide)
   cropped_img = center_cropper(img)
   resizer=torchvision.transforms.Resize((512,512))
   res_img=resizer(cropped_img)
   totensor=torchvision.transforms.ToTensor()
   img_tensor=torch.unsqueeze(totensor(res_img) ,dim=0)
   prediction=model(img_tensor).sigmoid()
   sum=torch.cumsum(prediction,dim=0)
   real_pre=torch.argmax(sum).cpu().numpy()
   print(real_pre)

0


In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
# import pytorch_lightning as pl
from google.colab import drive
# drive.mount('/content/drive')
# os.chdir('/content/drive/MyDrive/DRseg')
from pprint import pprint
from torch.utils.data import DataLoader

In [ ]:
# dataloader
import torchvision
from PIL import Image
from torch.utils.data import Dataset
DATASET_PATH='/content/drive/MyDrive/dataset/DDR-dataset/DR_grading'
CKPT_PATH="/content/drive/MyDrive/DRseg/SAVE/"
INPUT_SIZE=1024
# this is running on server
class GRADData(Dataset):  # 继承Dataset这个类
    def __init__(self,mode):
        self.img_path = DATASET_PATH+'/'+mode
        self.img_names = os.listdir(self.img_path)
        if mode=='train':
         self.label_path = DATASET_PATH+'/'+mode+'.txt'
        else:
         self.label_path = DATASET_PATH+'/valid.txt'
        self.label_dict = {}
        fopen = open(self.label_path)
        for line in fopen.readlines():
          line = str(line).replace("\n","") #remove \n
          self.label_dict[line.split(' ',1)[0]] = line.split(' ',1)[1]
        fopen.close()
        self.resizer=torchvision.transforms.Resize((INPUT_SIZE,INPUT_SIZE))
        self.totensor=torchvision.transforms.ToTensor()
    def __len__(self):
        return len(self.img_names)
    def __getitem__(self, item):
        img_name= self.img_names[item]
        img_path= os.path.join(self.img_path, self.img_names[item])
        img = Image.open(img_path) # fp -文件名 (字符串)，pathlib.Path对象或文件对象
        label = self.label_dict[img_name]
        w, h = img.size

        slide=min(h,w)
        center_cropper=torchvision.transforms.CenterCrop(slide)
        cropped_img = center_cropper(img)

        res_img=self.resizer(cropped_img)

        lr_flipped_img =self.totensor( self.resizer(cropped_img.transpose(Image.FLIP_LEFT_RIGHT) )) # 水平翻转
        td_flipped_img =self.totensor( self.resizer(cropped_img.transpose(Image.FLIP_TOP_BOTTOM) )) # 竖直翻转
        lr_td_flipped_img =self.totensor( self.resizer(cropped_img.transpose(Image.FLIP_TOP_BOTTOM).transpose(Image.FLIP_LEFT_RIGHT) )) # 水平+竖直翻转
        imgs=[self.totensor(self.resizer(cropped_img))]+[lr_flipped_img]+[td_flipped_img]+[ lr_td_flipped_img ]

        img_tensor=torch.stack(tuple(imgs))
        label_num=int(label)
        label_tensor=torch.zeros((4,1),dtype=torch.long)
        label_tensor[:,0]=label_num
        return img_tensor,label_tensor

In [ ]:
from torchvision.models import resnet34, ResNet34_Weights
from torch import nn
# from pytorch_lightning.callbacks import ModelCheckpoint
BATCH_SIZE=1
SAVE_PATH='/content/drive/MyDrive/DR_grading/trained_models'
# Using pretrained weights:
model=resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)
# 修改全连接层的输出
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 6)
model=model.cuda()
data = GRADData('train')
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)
grad_dataloader = DataLoader(data, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, drop_last=True)

EPOCH = 5
print('load net')
# model.load_state_dict(torch.load(SAVE_PATH+'/Resnet34_9epoch.pt'))
print('load success')
for epoch in range(EPOCH):
    print('begin {} epoch'.format(epoch))
    model.train() # model.train() 将 self.training = True，model.eval() 将 self.training = False，
    # 所以即便训练与测试共用一个模型，也能通过 self.training 来区分现在属于训练还是测试。它们用于切换模式。
    for i, (img, label) in enumerate(grad_dataloader):

        img = img.reshape(4*BATCH_SIZE,3,INPUT_SIZE,INPUT_SIZE)
        img=img.cuda()
        label = label.reshape(4*BATCH_SIZE).cuda()
        predict_label = model(img)# 前向传播 forward是在__call__中调用的，而__call__函数是在类的对象使用‘()’时被调用。

        loss = loss_func(predict_label, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    torch.save(model.state_dict(), SAVE_PATH+'/Resnet34_{}epoch.pt'.format(epoch))

load net
load success
begin 0 epoch
begin 1 epoch
begin 2 epoch
begin 3 epoch
begin 4 epoch


In [ ]:
loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
input=torch.tensor([[0.,0,0,0,1],
                    [0,1,0,0,0],
                    [0,0,1,0,0]])
print(input)
# target = torch.empty(3, dtype=torch.long).random_(5)
target=torch.tensor([4,1,2])
print(target)
output = loss(input, target)
print(output)

 # Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)


tensor([[0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])
tensor([4, 1, 2])
tensor(0.9048)


In [ ]:
from torchvision.models import resnet34, ResNet34_Weights
from torch import nn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, average_precision_score,precision_score,f1_score,recall_score
import numpy as np
SAVE_PATH='/content/drive/MyDrive/DR_grading/trained_models'
data = GRADData('test')
BATCH_SIZE=1
grad_dataloader = DataLoader(data, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, drop_last=True)
model=resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)
# 修改全连接层的输出
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 6)
model=model.cuda()

epoches='4'
print('load net')
model.load_state_dict(torch.load(SAVE_PATH+'/Resnet34_'+epoches+'epoch.pt'))
model.eval()
y_true=[0]
y_pred=[0]
with torch.no_grad():
 for i, (img, label) in enumerate(grad_dataloader):
   img=torch.squeeze(img,dim=0).cuda()
   prediction=model(img).sigmoid()
   sum=torch.cumsum(prediction,dim=0)[3]
   real_pre=torch.argmax(sum).cpu().numpy()
   y_true=np.hstack((y_true,(torch.argmax(label[0]).numpy())))
   y_pred=np.hstack((y_pred,real_pre))

C=confusion_matrix(y_true, y_pred)
print('------Weighted------')
print('Weighted precision', precision_score(y_true, y_pred, average='weighted'))
print('Weighted recall', recall_score(y_true, y_pred, average='weighted'))
print('Weighted f1-score', f1_score(y_true, y_pred, average='weighted'))
print('------Macro------')
print('Macro precision', precision_score(y_true, y_pred, average='macro'))
print('Macro recall', recall_score(y_true, y_pred, average='macro'))
print('Macro f1-score', f1_score(y_true, y_pred, average='macro'))
print('------Micro------')
print('Micro precision', precision_score(y_true, y_pred, average='micro'))
print('Micro recall', recall_score(y_true, y_pred, average='micro'))
print('Micro f1-score', f1_score(y_true, y_pred, average='micro'))

load net
------Weighted------
Weighted precision 1.0
Weighted recall 0.9399477806788512
Weighted f1-score 0.9690444145356663
------Macro------
Macro precision 0.3333333333333333
Macro recall 0.3133159268929504
Macro f1-score 0.3230148048452221
------Micro------
Micro precision 0.9399477806788512
Micro recall 0.9399477806788512
Micro f1-score 0.9399477806788512


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
